In [2]:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# class getResults:
    
#     def __init__(self):
        
#         self.results = []
        
        
#     def process_query(self,query):
        
#         stop_words_list = set(stopwords.words('english'))
#         relevant_terms = []
#         query_terms = word_tokenize(query)
#         relevant_terms = [x for x in query_terms if not x in self.stop_words_list]
        
#         return relevant_terms

#     def create_query_for_solr(self,relevant_terms):
        
#         query_types = []
#         qtype1 = ' '.join(relevant_terms)
#         docquery = 'doc_text:('+qtype1 +')'
        
# #         print(docquery)
#         titlequery = 'title:('+qtype1 +')^2.0 '
#         fullq = titlequery + docquery
# #         print(fullq)
        
#         mainquery = '"%s"' % " ".join([a.strip() for a in qtype1.split("\n") if a])
        
# #         print(mainquery)
        
#         return mainquery,fullq

        

# q,fullq = getResults().create_query_for_solr(['abv','bvc','poi'])

# print(q)
# print(fullq)

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#import chat_helper as _helper
import time 
# import spacy
import pysolr

class getResults:
    
    def __init__(self):
        
        self.results = []
        self.solr = pysolr.Solr('http://localhost:8983/solr/all_paragraphs_iiitd', timeout=10)
        
        
    def process_query(self,query):
        
#        start1 = time.time()
#        translations_and_syn = _helper.translations_and_synonyms(query)
#        end1 = time.time()
        
        stop_words_list = set(stopwords.words('english'))
        relevant_terms = []
        query_terms = word_tokenize(query)
        relevant_terms = [x for x in query_terms if not x in stop_words_list]
        
        return relevant_terms

    def create_query_for_solr(self,stop_words_removed,noun_entities=[]):
        
        query_types = []
        
        import re
       
            
        qtype1 = ' '.join(stop_words_removed)
        qtype2 = ' '.join(noun_entities)
        docquery = 'doc_text:('+qtype1 +')'
#         docquery2 = 'doc_text:('+qtype2 +')'


         
        valid1 = re.match('^[\w-]+$',docquery) is not None
#         valid2 =re.match('^[\w-]+$',docquery2) is not None

        if(not valid1):
            query_types.append(docquery)
#         query_types.append(docquery2)
#        print(docquery)
        
        titlequery = 'title:('+qtype1 +')^1.25'
        titlequery2 = 'title:('+qtype2 +')^2.0 '
        fullq = titlequery + docquery
#         fullq2 = titlequery2 + docquery2
#        print(fullq)

        valid3 =re.match('^[\w-]+$',fullq) is not None
#         valid4 =re.match('^[\w-]+$',fullq2) is not None
        if(not valid3):
            query_types.append(fullq)
#         if(valid4):
#         query_types.append(fullq2)
#        
        mainquery = '"%s"' % " ".join([a.strip() for a in qtype1.split("\n") if a])
#         mainquery2 = '"%s"' % " ".join([a.strip() for a in qtype2.split("\n") if a])
        
      
        valid5 =re.match('^[\w-]+$',mainquery) is not None
#         valid6 =re.match('^[\w-]+$',mainquery2) is not None
#        print(mainquery)
  
        
        if(not valid5):
            query_types.append(mainquery)
#         query_types.append(mainquery2)
        
        print('query types = '+str(query_types))
        return query_types
        
    def retrieve_results(self,query,filter_queries=[]):
        
        #noun_entities = _helper.extract_nouns(query)
        noun_entities = []
        relevant_terms = self.process_query(query)
        all_query_types = self.create_query_for_solr(relevant_terms,noun_entities)
        print(all_query_types)
    
        
        f = open('results.txt','w')
        
       
        for q in all_query_types:
            f.write('\n--------\n')
            f.write(q+'\n')
            result = self.solr.search(q,fq=filter_queries,fl='*,score')
            for j in result:
                try:
                    f.write(str(j)+'\n')
                except UnicodeEncodeError:
                    print('ERROR')
                    f.write(str(str(j).encode('utf-8'))+'\n')
                f.write('\n')
            self.results.append(result)
        return self.results
        
        
        
start2 = time.time()
getResults().retrieve_results('I wanted to inquire about the admission procedure')
end2 = time.time()


# print('2 = '+str(end2-start2))
#getResults().create_query_for_solr(['abv','bvc','poi'])

query types = ['doc_text:(I wanted inquire admission procedure)', 'title:(I wanted inquire admission procedure)^1.25doc_text:(I wanted inquire admission procedure)', '"I wanted inquire admission procedure"']
['doc_text:(I wanted inquire admission procedure)', 'title:(I wanted inquire admission procedure)^1.25doc_text:(I wanted inquire admission procedure)', '"I wanted inquire admission procedure"']


In [4]:
import pandas
filestr =str("qatest_pairs_new_factoids.csv")
allqas= pandas.read_csv(filestr,header=None)

In [5]:
qarr = allqas.iloc[:,0].values
aarr = allqas.iloc[:,1].values

In [35]:
count=0


itcount=5




#qarr[1]='when does Semester II start?'

#qarr[itcount] = 'what is the email address for mtech admission related queries'
#qarr[itcount] = 'How many seats  are there?'

print("==========================================================================")
print("Question => ",qarr[itcount])
print("Actual Ans=>",aarr[itcount])
print('----------')
count+=1
#     print()http://localhost:8888/notebooks/PycharmProjects/Chatbot/querygen.ipynb#

## Remove \u2756

rscount=0

resdict ={}
result1 = getResults().retrieve_results(qarr[itcount])[0]
result_from_q1 = []
for a in result1:
    rscount+=1
    print(" \nResult %s :- \n "%(rscount))
    result_from_q1.append(a)
    resdict[rscount]=a
# for a in getResults().retrieve_results(qarr[itcount])[2]:
    print(a)
#     print(a['title'],a['doc_text'])
    

# break
print("==========================================================================")


Question =>  Can we switch between the specializations?
Actual Ans=> Yes, with permission. Before applying ensure you are satisfying all the requirement for that specialization
----------
query types = ['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']
['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']
 
Result 1 :- 
 
{'url': ['http://iiitd.ac.in/academics/mtech/ece'], 'title_orig': ['General Requirements'], 'doc_text': ['A student may be enrolled in “M.Tech (ECE) with specialization in <area>” or “M.Tech (ECE)”. Specialization areas are as notified/ announced. A student enrolled in a specialization can move to M.Tech (ECE) at any point by informing suitably. A student can move from M.Tech (ECE) to a specialization only if permitted by the PG Committee. M.Tech (ECE) 

In [36]:
print("==========================================================================")
print("Question => ",qarr[itcount])
print("Actual Ans=>",aarr[itcount])
print('----------')
count+=1
#     print()

rscount=0
result2 = getResults().retrieve_results(qarr[itcount])[1]
result_from_q2 = []
for a in result2:
# for a in getResults().retrieve_results(qarr[itcount])[2]:
    rscount+=1
    result_from_q2.append(a)
    print(" \nResult %s :-\n "%(rscount))
    print(a)
#     print(a['title'],a['doc_text'])
    

# break
print("==========================================================================")


Question =>  Can we switch between the specializations?
Actual Ans=> Yes, with permission. Before applying ensure you are satisfying all the requirement for that specialization
----------
query types = ['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']
['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']
 
Result 1 :-
 
{'url': ['http://iiitd.ac.in/education/mtech/mobile-comp'], 'title_orig': ['Specialization Requirements'], 'doc_text': ['In addition to the core course requirements for the M.Tech. program, to obtain specialization in Mobile Computing a student must: Complete at least 16 credits of courses in the overall area of Mobile Computing. Do his/her thesis or Scholarly paper credits in Data Engineering. The advisor will certify this fact. If a course appears in m

In [37]:
# Remove Change History
print("==========================================================================")
print("Question => ",qarr[itcount])
print("Actual Ans=>",aarr[itcount])
print('----------')
count+=1
#     print()
rscount=0
for a in getResults().retrieve_results(qarr[itcount])[2]:
# for a in getResults().retrieve_results(qarr[itcount])[2]:
    rscount+=1
    print(" \nResult %s :-\n "%(rscount))
    print(a)
#     print(a['title'],a['doc_text'])
    

# break
print("==========================================================================")



Question =>  Can we switch between the specializations?
Actual Ans=> Yes, with permission. Before applying ensure you are satisfying all the requirement for that specialization
----------
query types = ['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']
['doc_text:(Can switch specializations ?)', 'title:(Can switch specializations ?)^1.25doc_text:(Can switch specializations ?)', '"Can switch specializations ?"']


In [38]:
top_results_1 = result_from_q1[0:5]
top_results_2 = result_from_q2[0:5]

final_results = []

for a in range(len(top_results_1)):
   
   for b in range(len(top_results_2)):
       
       if top_results_1[a]['id'] == top_results_2[b]['id']:
            final_results.append([top_results_1[a],top_results_2[b]])

#print(len(final_results))
#print(final_results)
import numpy as np


final_res=""

if len(final_results) == 1:
    final_res=final_results[0][0]
    print(final_results[0][0])

elif len(final_results)>1:
   
    final_scores = []
    for i in range(len(final_results)):
        final_scores.append(final_results[i][0]['score']+final_results[i][1]['score'])

    index = np.argmax(final_scores)
    final_res=final_results[index][0]
    print(final_results[index][0])

else:
    final_res=top_results_1[0]
#     if top_results_1[0]['score']>top_results_2[0]['score']:
#         final_res=top_results_1[0]
#         print(top_results_1[0])
#     else:
#         final_res=top_results_2[0]
#         print(top_results_2[0])

In [39]:
# count=0
# for a in qarr:
#     print("==========================================================================")
#     print(a)
#     print("Ans=>",aarr[count])
#     count+=1
# #     print()
#     for b in getResults().retrieve_results(a)[0]:
#         print(b)
        
#     break
#     print("==========================================================================")
   

In [40]:
final_res['doc_text']

['A student may be enrolled in “M.Tech (ECE) with specialization in <area>” or “M.Tech (ECE)”. Specialization areas are as notified/ announced. A student enrolled in a specialization can move to M.Tech (ECE) at any point by informing suitably. A student can move from M.Tech (ECE) to a specialization only if permitted by the PG Committee. M.Tech (ECE) may be done with a thesis, or without a thesis but with a scholarly paper. In both options, students have to do certain amount of coursework. In addition, students doing M.Tech with thesis will have to do a thesis. Students in scholarly paper option will do instead of a thesis a scholarly paper, and will have to do additional courses. A student doing M.Tech with scholarly paper has multiple options for completing the scholarly paper requirement: a regular scholarly paper, industrial project and capstone project. The overall credit requirement for the M.Tech is 48 credits. Requirements for thesis and scholarly paper options are as follows: 

In [41]:
resdict[1]

{'_version_': 1629242147478700041,
 'branch': ['ece'],
 'degree': ['mtech'],
 'doc_text': ['A student may be enrolled in “M.Tech (ECE) with specialization in <area>” or “M.Tech (ECE)”. Specialization areas are as notified/ announced. A student enrolled in a specialization can move to M.Tech (ECE) at any point by informing suitably. A student can move from M.Tech (ECE) to a specialization only if permitted by the PG Committee. M.Tech (ECE) may be done with a thesis, or without a thesis but with a scholarly paper. In both options, students have to do certain amount of coursework. In addition, students doing M.Tech with thesis will have to do a thesis. Students in scholarly paper option will do instead of a thesis a scholarly paper, and will have to do additional courses. A student doing M.Tech with scholarly paper has multiple options for completing the scholarly paper requirement: a regular scholarly paper, industrial project and capstone project. The overall credit requirement for the 

In [42]:
# E:\berttrained\bert-master
import os
os.chdir("E:\\berttrained\\bert-master")
# !dir

contextdict ={}


# strcontex = ''.join(resdict[1]['title_orig']) + " "+ ''.join(resdict[1]['doc_text'])

strcontex = ''.join(final_res['title_orig']) + " "+ ''.join(final_res['doc_text'])

ques = qarr[itcount]

print(ques)
print(strcontex)

strcontex = strcontex.replace("'","")
# print(strcontex)

strtitle="IIITD"




contextdict["context"] =strcontex
ansdict ={}
ansdict["answers"] = [{'answer_start': 1, 'text': ''}, {'answer_start': 1, 'text': ''},{'answer_start': 1, 'text': ''}] 

                       
ansdict["id"] ='1'
ansdict["question"]=ques

ansdict


newdict={}
newdict["qas"] =[ansdict]

contextdict["qas"] = newdict["qas"]
contextdict

maindict ={}
maindict["title"] = strtitle
maindict["paragraphs"] = [contextdict]

maindict

finmaindict={}
finmaindict["data"]=[maindict]
finmaindict

import json
with open('questest.json','w') as json_data:
    json.dump(finmaindict,json_data)
    
print("****exported")


Can we switch between the specializations?
General Requirements A student may be enrolled in “M.Tech (ECE) with specialization in <area>” or “M.Tech (ECE)”. Specialization areas are as notified/ announced. A student enrolled in a specialization can move to M.Tech (ECE) at any point by informing suitably. A student can move from M.Tech (ECE) to a specialization only if permitted by the PG Committee. M.Tech (ECE) may be done with a thesis, or without a thesis but with a scholarly paper. In both options, students have to do certain amount of coursework. In addition, students doing M.Tech with thesis will have to do a thesis. Students in scholarly paper option will do instead of a thesis a scholarly paper, and will have to do additional courses. A student doing M.Tech with scholarly paper has multiple options for completing the scholarly paper requirement: a regular scholarly paper, industrial project and capstone project. The overall credit requirement for the M.Tech is 48 credits. Requir

In [43]:
import os
os.chdir("E:\\berttrained\\bert-master")
# !dir

import tensorflow as tf


# Important - Clear flags before running bert
def del_all_flags(FLAGS):
    # Function to Clear flags before running bert
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

# Calling the clear Flags to clear tensorflow flags before calling Bert
del_all_flags(tf.flags.FLAGS)

# Running Bert for predictions
%run -i run_squad.py  --vocab_file=vocab.txt --bert_config_file=bert_config.json --init_checkpoint=out -do_predict=True --predict_file=questest.json --output_dir=outcheck 

INFO:tensorflow:Using config: {'_model_dir': 'outcheck', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002D48034F5F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_

INFO:tensorflow:input_ids: 101 2064 2057 6942 2090 1996 28031 2015 1029 102 2236 5918 1037 3076 2089 2022 8302 1999 1523 1049 1012 6627 1006 14925 2063 1007 2007 28031 1999 1026 2181 1028 1524 2030 1523 1049 1012 6627 1006 14925 2063 1007 1524 1012 28031 2752 2024 2004 19488 1013 2623 1012 1037 3076 8302 1999 1037 28031 2064 2693 2000 1049 1012 6627 1006 14925 2063 1007 2012 2151 2391 2011 21672 4848 8231 1012 1037 3076 2064 2693 2013 1049 1012 6627 1006 14925 2063 1007 2000 1037 28031 2069 2065 7936 2011 1996 18720 2837 1012 1049 1012 6627 1006 14925 2063 1007 2089 2022 2589 2007 1037 9459 1010 2030 2302 1037 9459 2021 2007 1037 12683 3259 1012 1999 2119 7047 1010 2493 2031 2000 2079 3056 3815 1997 2607 6198 1012 1999 2804 1010 2493 2725 1049 1012 6627 2007 9459 2097 2031 2000 2079 1037 9459 1012 2493 1999 12683 3259 5724 2097 2079 2612 1997 1037 9459 1037 12683 3259 1010 1998 2097 2031 2000 2079 3176 5352 1012 1037 3076 2725 1049 1012 6627 2007 12683 3259 2038 3674 7047 2005 7678 199

INFO:tensorflow:token_is_max_context: 10:False 11:False 12:False 13:False 14:False 15:False 16:False 17:False 18:False 19:False 20:False 21:False 22:False 23:False 24:False 25:False 26:False 27:False 28:False 29:False 30:False 31:False 32:False 33:False 34:False 35:False 36:False 37:False 38:False 39:False 40:False 41:False 42:False 43:False 44:False 45:False 46:False 47:False 48:False 49:False 50:False 51:False 52:False 53:False 54:False 55:False 56:False 57:False 58:False 59:False 60:False 61:False 62:False 63:False 64:False 65:False 66:False 67:False 68:False 69:False 70:False 71:False 72:False 73:False 74:False 75:False 76:False 77:False 78:False 79:False 80:False 81:False 82:False 83:False 84:False 85:False 86:False 87:False 88:False 89:False 90:False 91:False 92:False 93:False 94:False 95:False 96:False 97:False 98:False 99:False 100:False 101:False 102:False 103:False 104:False 105:False 106:False 107:False 108:False 109:False 110:False 111:False 112:False 113:False 114:False 11

INFO:tensorflow:token_to_orig_map: 10:190 11:191 12:192 13:192 14:193 15:194 16:195 17:196 18:197 19:198 20:198 21:199 22:200 23:201 24:202 25:203 26:204 27:205 28:206 29:207 30:207 31:208 32:209 33:210 34:211 35:212 36:213 37:214 38:215 39:216 40:217 41:218 42:219 43:220 44:221 45:221 46:222 47:223 48:224 49:225 50:226 51:227 52:228 53:229 54:229 55:230 56:231 57:232 58:233 59:234 60:235 61:235 62:236 63:236 64:237 65:238 66:239 67:240 68:241 69:242 70:243 71:244 72:245 73:246 74:247 75:248 76:249 77:250 78:251 79:252 80:252 81:253 82:254 83:255 84:256 85:257 86:258 87:258 88:258 89:259 90:260 91:261 92:262 93:262 94:263 95:264 96:265 97:266 98:267 99:267 100:268 101:269 102:270 103:271 104:272 105:273 106:274 107:275 108:276 109:277 110:277 111:278 112:278 113:279 114:280 115:281 116:282 117:283 118:284 119:285 120:286 121:287 122:288 123:289 124:290 125:291 126:291 127:292 128:293 129:294 130:295 131:295 132:296 133:297 134:298 135:299 136:300 137:301 138:302 139:303 140:304 141:305

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:input_ids: 101 2064 2057 6942 2090 1996 28031 2015 1029 102 9459 2089 2022 8897 2875 9459 2030 12683 3259 4414 1010 2065 4844 2011 1996 18720 2278 1012 2169 1049 1012 6627 1006 14925 2063 1007 3076 2038 2000 13225 1996 4563 2607 9095 1012 2005 1037 28031 1010 2023 9095 2003 8510 2011 7678 2035 1996 4563 5352 2005 2008 28031 1012 2005 1049 1012 6627 1006 14925 2063 1007 2302 28031 1010 2023 2064 2022 8510 2011 7678 2260 6495 2013 1996 4563 5352 1997 2151 1997 1996 28031 2015 1012 2035 2060 5352 2024 11322 24653 1012 1999 11322 24653 1010 2012 2087 1018 6495 1997 1523 2981 2817 1524 1998 1018 6495 1997 1523 3576 2622 1524 2064 2022 2579 1012 3784 2607 2089 2022 7936 2000 4236 2004 2981 2817 1013 3576 2622 2007 6656 1997 18720 2278 1012 5918 2005 28031 2005 1523 1049 1012 6627 1999 14925 2063 2007 28031 1999 1026 2181 1028 1524 1996 3076 2442 1024 3143 1996 4563 5352 1997 1996 28031 2181 1012 3143 2012 2560 2322 6495 1997 5352 1999 1996 4217 28031 2181 1010 2164 1996 4563 

INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = cls/squad/output_weights:0, shape = (2, 768

In [44]:
import json
import os
os.chdir("E:\\berttrained\\bert-master")

predpath = "outcheck/"+"predictions.json"
preddict={}
with open(predpath) as json_data:
    preddict = json.load(json_data)
print(preddict['1'])


answer = preddict['1']
answerurl = final_res['url'][0]




if permitted by the PG Committee


In [45]:
print(answerurl)

http://iiitd.ac.in/academics/mtech/ece


In [46]:

# npredpath = "outcheck/"+"nbest_predictions.json"
# npreddict={}
# with open(npredpath) as json_data:
#     npreddict = json.load(json_data)
# print(npreddict)


